In [1]:
!pwd

/aiffel/aiffel/Aimmo


In [2]:
%cd mmdetection

#완디비 설치
#!pip install -q --upgrade wandb
#!pip3 install openmim
#!mim install mmcv-full
#!pip install -e .

/aiffel/aiffel/Aimmo/mmdetection


In [3]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0.13.6


In [4]:
# # 완디비 로그인
wandb.login(key= '')

wandb: Currently logged in as: cychuckpark (yolo_x). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [5]:
wandb.init(project="test-project", entity="yolo_x")

In [6]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'bus', 'truck','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_txt')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [7]:
config_file = 'configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg = Config.fromfile(config_file)

cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/'

cfg.data.train.type = 'Aimmo'
cfg.data.train.data_root = 'data/'
cfg.data.train.ann_file = 'middle_train.txt'
cfg.data.train.img_prefix = 'image'


cfg.data.val.type = 'Aimmo'
cfg.data.val.data_root = 'data/'
cfg.data.val.ann_file = 'middle_val.txt'
cfg.data.val.img_prefix = 'image'


cfg.data.test.type = 'Aimmo'
cfg.data.test.data_root = 'data/'
cfg.data.test.ann_file = 'test/middle_test.txt'
cfg.data.test.img_prefix = 'test/image'

cfg.model.roi_head.bbox_head.num_classes = 4

cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg.work_dir = 'middle'

cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.evaluation.metric='mAP'

cfg.log_config.interval = 100
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1
#cfg.evaluation.save_best='auto'
cfg.data.train.pipeline[2].img_scale=[(960,512)]#['Resize']
cfg.train_pipeline[2].img_scale=[(960,512)]
cfg.test_pipeline[1].img_scale=[(960,512)]
cfg.data.val.pipeline[1].img_scale = [(960,512)]
cfg.data.test.pipeline[1].img_scale = [(960,512)]
cfg.runner.max_epochs = 15

cfg.data.samples_per_gpu = 14 # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 0 # Worker to pre-fetch data for each single GPU
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [8]:
cfg.model.rpn_head.anchor_generator

{'type': 'AnchorGenerator',
 'scales': [8],
 'ratios': [0.5, 1.0, 2.0],
 'strides': [4, 8, 16, 32, 64]}

In [9]:
cfg.model.rpn_head.anchor_generator['scales'] = [4]
cfg.model.rpn_head.anchor_generator['strides'] = [2,8,16,32,64]

In [10]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'test-project', 'entity' : 'yolo_x', 'name' : 'middle'})] 

In [11]:
[build_dataset(cfg.data.val)]

/aiffel/aiffel/Aimmo/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


[
 Aimmo Train dataset with number of images 2323, and instance counts: 
 +----------+-------+----------+-------+-----------+-------+----------------+-------+---------------+-------+
 | category | count | category | count | category  | count | category       | count | category      | count |
 +----------+-------+----------+-------+-----------+-------+----------------+-------+---------------+-------+
 | 0 [car]  | 13124 | 1 [bus]  | 1786  | 2 [truck] | 3084  | 3 [pedestrian] | 3063  | -1 background | 0     |
 +----------+-------+----------+-------+-----------+-------+----------------+-------+---------------+-------+]

In [12]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-12-08 10:45:38,264 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-08 10:45:38,422 - mmdet - INFO - load checkpoint from local path: checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2022-12-08 10:45:38,600 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([16, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([16]).
20

2022-12-08 10:50:17,053 - mmdet - INFO - Epoch [1][100/1494]	lr: 3.976e-03, eta: 16:53:42, time: 2.726, data_time: 1.196, memory: 11276, loss_rpn_cls: 0.0474, loss_rpn_bbox: 0.0308, loss_cls: 0.3012, acc: 91.5008, loss_bbox: 0.2171, loss: 0.5964
2022-12-08 10:54:45,324 - mmdet - INFO - Epoch [1][200/1494]	lr: 7.972e-03, eta: 16:41:07, time: 2.683, data_time: 1.092, memory: 11276, loss_rpn_cls: 0.0175, loss_rpn_bbox: 0.0197, loss_cls: 0.1841, acc: 93.6959, loss_bbox: 0.2336, loss: 0.4550
2022-12-08 11:00:03,397 - mmdet - INFO - Epoch [1][300/1494]	lr: 1.197e-02, eta: 17:35:04, time: 3.180, data_time: 1.611, memory: 11276, loss_rpn_cls: 0.0142, loss_rpn_bbox: 0.0183, loss_cls: 0.1643, acc: 94.3209, loss_bbox: 0.2241, loss: 0.4209
2022-12-08 11:05:29,798 - mmdet - INFO - Epoch [1][400/1494]	lr: 1.596e-02, eta: 18:07:02, time: 3.264, data_time: 1.607, memory: 11276, loss_rpn_cls: 0.0122, loss_rpn_bbox: 0.0170, loss_cls: 0.1612, acc: 94.4255, loss_bbox: 0.2226, loss: 0.4129
2022-12-08 11:10

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.0 task/s, elapsed: 460s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-08 12:11:48,583 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 37625 | 0.953  | 0.907 |
| bus        | 1786  | 5871  | 0.892  | 0.805 |
| truck      | 3084  | 10763 | 0.874  | 0.734 |
| pedestrian | 3063  | 13976 | 0.771  | 0.595 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.760 |
+------------+-------+-------+--------+-------+
2022-12-08 12:11:48,671 - mmdet - INFO - Epoch(val) [1][2323]	AP50: 0.7600, mAP: 0.7603
2022-12-08 12:17:09,123 - mmdet - INFO - Epoch [2][100/1494]	lr: 2.000e-02, eta: 17:07:31, time: 3.199, data_time: 1.671, memory: 11276, loss_rpn_cls: 0.0075, loss_rpn_bbox: 0.0132, loss_cls: 0.1169, acc: 95.8129, loss_bbox: 0.1843, loss: 0.3218
2022-12-08 12:22:31,947 - mmdet - INFO - Epoch [2][200/1494]	lr: 2.000e-02, eta: 17:08:01, time: 3.228, data_time: 1.682, memory: 11276, loss_rp

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.0 task/s, elapsed: 468s, ETA:     0s
---------------iou_thr: 0.5---------------


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 4.9 task/s, elapsed: 472s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-09 07:46:11,823 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 20295 | 0.964  | 0.946 |
| bus        | 1786  | 3285  | 0.942  | 0.914 |
| truck      | 3084  | 6772  | 0.924  | 0.876 |
| pedestrian | 3063  | 7754  | 0.849  | 0.757 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.873 |
+------------+-------+-------+--------+-------+
2022-12-09 07:46:11,910 - mmdet - INFO - Epoch(val) [14][2323]	AP50: 0.8730, mAP: 0.8731
2022-12-09 07:51:40,375 - mmdet - INFO - Epoch [15][100/1494]	lr: 2.000e-04, eta: 1:11:50, time: 3.279, data_time: 1.682, memory: 11276, loss_rpn_cls: 0.0025, loss_rpn_bbox: 0.0080, loss_cls: 0.0564, acc: 97.8425, loss_bbox: 0.1181, loss: 0.1850
2022-12-09 07:57:04,469 - mmdet - INFO - Epoch [15][200/1494]	lr: 2.000e-04, eta: 1:06:41, time: 3.241, data_time: 1.672, memory: 11276, loss_r

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 2323/2323, 5.0 task/s, elapsed: 465s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-09 09:15:24,643 - mmdet - INFO - 
+------------+-------+-------+--------+-------+
| class      | gts   | dets  | recall | ap    |
+------------+-------+-------+--------+-------+
| car        | 13124 | 20451 | 0.964  | 0.947 |
| bus        | 1786  | 3261  | 0.941  | 0.913 |
| truck      | 3084  | 6776  | 0.925  | 0.876 |
| pedestrian | 3063  | 7827  | 0.848  | 0.754 |
+------------+-------+-------+--------+-------+
| mAP        |       |       |        | 0.872 |
+------------+-------+-------+--------+-------+
2022-12-09 09:15:24,742 - mmdet - INFO - Epoch(val) [15][2323]	AP50: 0.8720, mAP: 0.8723


learning_rate,▄█████████████████████▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁▃▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇█▇██████████████
train/loss,█▆▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_bbox,█▇▆▅▅▅▄▄▄▄▄▃▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
train/loss_cls,█▆▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▆▅▄▄▄▄▃▃▃▃▂▃▃▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_rpn_cls,█▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/AP50,▁▄▅▅▆▇▇▇███████
val/mAP,▁▄▅▅▆▇▇▇███████
learning_rate,0.0002
